In [13]:
import pandas as pd

data = pd.read_csv('lda_xtest_datset.csv').rename(columns={'Unnamed: 0': 'index_0'})
golden = pd.read_csv('lda_final_dataset.csv')['index'].reset_index()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1854 entries, 0 to 1853
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   index_0                 1854 non-null   int64  
 1   cosine_constitution     1854 non-null   float64
 2   cosine_crpc             1854 non-null   float64
 3   cosine_ipc              1854 non-null   float64
 4   Topic_0                 1854 non-null   float64
 5   Topic_1                 1854 non-null   float64
 6   Topic_2                 1854 non-null   float64
 7   Topic_3                 1854 non-null   float64
 8   Perc_Dominant_Topic     1854 non-null   float64
 9   Dominant_Topic_Topic 0  1854 non-null   float64
 10  Dominant_Topic_Topic 1  1854 non-null   float64
 11  Dominant_Topic_Topic 2  1854 non-null   float64
 12  Dominant_Topic_Topic 3  1854 non-null   float64
 13  label                   1854 non-null   int64  
 14  et_prediction           1854 non-null   

In [17]:
combined = data.merge(golden, left_on='index_0', right_on='level_0', how='inner')
combined

,index_0,cosine_constitution,cosine_crpc,cosine_ipc,Topic_0,Topic_1,Topic_2,Topic_3,Perc_Dominant_Topic,Dominant_Topic_Topic 0,Dominant_Topic_Topic 1,Dominant_Topic_Topic 2,Dominant_Topic_Topic 3,label,et_prediction,stacked_prediction,level_0,index
0,457,-0.101787,-999.000000,-999.000000,0.001060,0.000067,0.078062,0.920811,0.92,0.0,0.0,0.0,1.0,1,1,1,457,228
1,7537,-0.052895,-999.000000,-999.000000,0.054419,0.476009,0.126036,0.343536,0.48,0.0,1.0,0.0,0.0,0,0,0,7537,4649
2,6384,0.027554,-0.000433,-999.000000,0.026167,0.108493,0.088267,0.777073,0.78,0.0,0.0,0.0,1.0,1,1,1,6384,3944
3,5605,-999.000000,-0.096937,-0.044178,0.000322,0.999042,0.000317,0.000319,1.00,0.0,1.0,0.0,0.0,0,0,0,5605,3522
4,4655,0.028142,0.028187,-999.000000,0.000264,0.609246,0.102401,0.288088,0.61,0.0,1.0,0.0,0.0,1,1,1,4655,2913
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1849,7920,-999.000000,-999.000000,-999.000000,0.000516,0.000506,0.000513,0.998464,1.00,0.0,0.0,0.0,1.0,0,0,0,7920,4898
1850,8502,-999.000000,-999.000000,-999.000000,0.000258,0.000261,0.007124,0.992357,0.99,0.0,0.0,0.0,1.0,1,0,0,8502,5368
1851,2687,-0.006214,0.041286,-999.000000,0.013834,0.037376,0.260677,0.688112,0.69,0.0,0.0,0.0,1.0,0,0,0,2687,1629
1852,8010,-999.000000,-999.000000,-999.000000,0.000656,0.000678,0.000659,0.998007,1.00,0.0,0.0,0.0,1.0,1,0,1,8010,4953


In [20]:
combined['index'].value_counts()

3595    18
624     13
602     11
92       9
4344     9
        ..
4759     1
4169     1
4214     1
2535     1
1803     1
Name: index, Length: 1573, dtype: int64

In [22]:
from torch.utils.data import Dataset
import logging

class MajorVoting(Dataset):
    def __init__(
            self, 
            df,
            target_name,
            target_transform=None,  # this is should be paired with 'target_transform_groupby'
            target_transform_groupby=None  # this is should be paired with 'target_transform'
        ):
        self.dataframe = df
        self.target_transform = target_transform
        self.target_transform_groupby = target_transform_groupby
        self.target_name = target_name

        self.save_majority_articles = pd.DataFrame()
        self.augemented_data_source = ['constitution_article', 'crpc_article', 'ipc_article']


        if self.target_transform_groupby:
            self.target_transform_groupby_index = self.dataframe[self.target_transform_groupby].unique()
        else:
            self.target_transform_groupby_index = None

    def return_saved_majority_articles(self, idx):
        group = self.dataframe.groupby(by=[self.target_transform_groupby]).get_group(idx)
        major = group[self.target_name].mean()  # easiest way to get major voting is apply .mean() 
        # (accroding to the pandas documentation, we are forced to apply agg function, otherwise we get a generator object)
        # binary target, mean produced continuous result between 0 and 1
        target_value = 1 if major >= 0.5 else 0


        filtered_by_target = group[group[self.target_name] == major]
        text_from_augmented_source = {}
        for source in self.augemented_data_source:
            text_from_augmented_source[source] = filtered_by_target[source].dropna().values

        return text_from_augmented_source

    def major_voting_transformation(self, idx):
        group = self.dataframe.groupby(by=[self.target_transform_groupby]).get_group(idx)
        major = group[self.target_name].mean()  # easiest way to get major voting is apply .mean() 
        # (accroding to the pandas documentation, we are forced to apply agg function, otherwise we get a generator object)
        # binary target, mean produced continuous result between 0 and 1
        return_group = group.iloc[0].copy()
        return_group[f'major_{self.target_name}'] = 1 if major >= 0.5 else 0
        return return_group

    def __getitem__(self, idx):
        if self.target_transform == 'major_voting':
            row = self.major_voting_transformation(idx)
            label = row[f'major_{self.target_name}']
            extra_info = self.return_saved_majority_articles(idx)
        else:
            row = self.dataframe.iloc[idx]
            label = self.dataframe.loc[idx, self.target_name]
            extra_info = None
            
        return row, label, extra_info

In [25]:
majority_voting_dataset = MajorVoting(
    combined, 
    'et_prdiction', 
    target_transform='major_voting', 
    target_transform_groupby='index'
    )

In [ ]:
a, b, c = majority_voting_dataset[0]

In [34]:
combined.groupby(by=['index']).mean()[['et_prediction', 'stacked_prediction']].describe()

,et_prediction,stacked_prediction
count,1573.000000,1573.000000
mean,0.343611,0.324857
std,0.474226,0.467791
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,1.000000,1.000000
max,1.000000,1.000000


In [35]:
first = combined.groupby(by=['index']).first()
first

,index_0,cosine_constitution,cosine_crpc,cosine_ipc,Topic_0,Topic_1,Topic_2,Topic_3,Perc_Dominant_Topic,Dominant_Topic_Topic 0,Dominant_Topic_Topic 1,Dominant_Topic_Topic 2,Dominant_Topic_Topic 3,label,et_prediction,stacked_prediction,level_0
index,,,,,,,,,,,,,,,,,
1,5,-0.005294,-999.000000,-999.000000,0.000135,0.000150,0.423930,0.575785,0.58,0.0,0.0,0.0,1.0,0,0,0,5
3,15,0.056916,-0.091344,0.018679,0.000221,0.716355,0.079194,0.204229,0.72,0.0,1.0,0.0,0.0,1,1,1,15
6,18,-0.099436,-999.000000,-999.000000,0.058391,0.191868,0.357055,0.392686,0.39,0.0,0.0,0.0,1.0,1,0,0,18
8,21,-0.093746,-999.000000,-999.000000,0.000098,0.000097,0.392683,0.607122,0.61,0.0,0.0,0.0,1.0,0,0,0,21
9,23,-0.051701,-999.000000,-999.000000,0.000333,0.263580,0.000331,0.735756,0.74,0.0,0.0,0.0,1.0,1,0,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6050,9241,-999.000000,-999.000000,-999.000000,0.000260,0.000263,0.000263,0.999214,1.00,0.0,0.0,0.0,1.0,0,0,0,9241
6059,9250,-999.000000,-999.000000,-0.015621,0.000302,0.892902,0.000301,0.106495,0.89,0.0,1.0,0.0,0.0,0,0,0,9250
6066,9257,-999.000000,-999.000000,-999.000000,0.000163,0.135015,0.000164,0.864659,0.86,0.0,0.0,0.0,1.0,1,1,1,9257


In [40]:
from sklearn.metrics import f1_score, classification_report

print(classification_report(first['label'].values, first['stacked_prediction'].values))

              precision    recall  f1-score   support

           0       0.73      0.80      0.76       970
           1       0.61      0.52      0.56       603

    accuracy                           0.69      1573
   macro avg       0.67      0.66      0.66      1573
weighted avg       0.68      0.69      0.69      1573



In [41]:
print(classification_report(first['label'].values, first['et_prediction'].values))

              precision    recall  f1-score   support

           0       0.73      0.77      0.75       970
           1       0.59      0.53      0.56       603

    accuracy                           0.68      1573
   macro avg       0.66      0.65      0.65      1573
weighted avg       0.67      0.68      0.68      1573

